In [1]:
# Step 1: Install Required Libraries
!pip install sentence_transformers
!pip install weaviate-client==4.5.4
!pip install langchain tiktoken pypdf transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
# Step 2: Upload your WhatsApp chat file (e.g., chat_group.txt)
from google.colab import files
uploaded = files.upload()  # Choose your 'chat_group.txt' file

Saving chat_group.txt to chat_group.txt


In [5]:
# Step 3: Parse the WhatsApp chat file
import re
from datetime import datetime

# Load file content
with open('chat_group.txt', 'r', encoding='utf-8') as f:
    raw_lines = f.readlines()

structured_data = []
your_name = "😊 😴💤"  # Define your WhatsApp name (emoji or text)
replace_with = "You"

# Regular expression to match WhatsApp message lines
pattern = re.compile(r'^(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}) - (.*?): (.*)')

for line in raw_lines:
    match = pattern.match(line)
    if match:
        date_str, time_str, sender, message = match.groups()
        # Replace your name with "You"
        sender = replace_with if sender.strip() == your_name else sender.strip()
        # Create a combined timestamp
        timestamp = datetime.strptime(f'{date_str} {time_str}', '%d/%m/%Y %H:%M')
        structured_data.append({
            'timestamp': timestamp.isoformat(),
            'sender': sender,
            'message': message.strip()
        })

print(f"Parsed {len(structured_data)} messages.")
print(structured_data[151:155])  # Preview first 5 messages

Parsed 20333 messages.
[{'timestamp': '2021-09-16T19:44:00', 'sender': 'Hamid Awan Vms4', 'message': 'Oh sorry Ishfaq my bro 😘'}, {'timestamp': '2021-09-16T19:59:00', 'sender': 'Usman Mujtaba Vms4', 'message': '😂😂'}, {'timestamp': '2021-09-16T23:23:00', 'sender': 'Muiz 14punjab Vms4', 'message': 'Ye kya hai bc'}, {'timestamp': '2021-09-16T23:23:00', 'sender': 'Inam Ul Haq Vms4', 'message': 'Yeh to mujtaba ki awaaz lgrai'}]


In [6]:
# Step 4: Convert structured data to LangChain Documents
from langchain.schema import Document

documents = []
for msg in structured_data:
    metadata = {"sender": msg["sender"], "timestamp": msg["timestamp"]}
    doc = Document(page_content=msg["message"], metadata=metadata)
    documents.append(doc)

print(f"Converted {len(documents)} messages into LangChain Documents.")

Converted 20333 messages into LangChain Documents.


In [7]:
# Step 5: (Optional) Split long documents into smaller chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = splitter.split_documents(documents)
print(f"After splitting: {len(split_documents)} chunks.")

After splitting: 20346 chunks.


In [9]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [10]:
# Step 6: Setup HuggingFace Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

<ipython-input-10-3aec2eda73f4>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [11]:
# Step 7: Connect to Weaviate Cloud using the v4 client
import weaviate
from weaviate.auth import AuthApiKey

# Create an auth object with your API key
auth = AuthApiKey(api_key="riiVJc8O3FMtHS06Jw464YbLaxzv4XjgQfdX")

# Pass the instance URL as a positional argument (do not use keyword args like url or scheme)
client = weaviate.Client(
    "https://ujgcimojshidjga1y3j2g.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_client_secret=auth
)

In [12]:
# Step 8: Create the Weaviate Vector Store and Upload Your Documents
from langchain.vectorstores.weaviate import Weaviate as LCWeaviate

vectorstore = LCWeaviate.from_documents(
    documents=split_documents,
    embedding=embedding,
    client=client,
    index_name="WhatsappChat"  # This is the Weaviate class name
)

print("Documents successfully uploaded to Weaviate!")

Documents successfully uploaded to Weaviate!


In [13]:
# Step 9: Set up your LLM pipeline (using Falcon-7B-Instruct as an example)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-13-828d9b785807>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [26]:
# Step 10: Set up a RetrievalQA chain and run a sample query
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [28]:
# Sample query
query = "protest is being done by which political party in the given chat?"
response = qa_chain(query)

print(response["result"])
# for doc in response["source_documents"]:
#     print(doc.metadata)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Protest kdr ha?

Protest kdr ha?

Govt is protesting for them

Question: protest is being done by which political party in the given chat?
Helpful Answer:

The political party that is protesting in the chat is the opposition party.
